In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
pkg = mp.jupyter.open_package()
#pkg = mp.jupyter.open_source_package()
pkg

In [3]:
ci = pkg.resource('checkins').dataframe()
ci.head()

,user_id,id,location,feeling,start_date,submit_date,network_id,form,clientid,tablecloth_link,qr_code_link
0,901,,ElCajon,heart eyes,2017-10-25 23:29:23,2017-10-25 23:29:23,,Checkin,,,
1,902,,ElCajon,smiling face,2017-10-25 23:30:37,2017-10-25 23:30:37,,Checkin,,,
2,81,,ElCajon,heart eyes,2017-10-25 23:32:42,2017-10-25 23:32:42,,Checkin,,,
3,1004,,ElCajon,screaming in fear,2017-10-25 23:46:01,2017-10-25 23:46:01,,Checkin,,,
4,1009,,ElCajon,smiling face,2017-10-25 23:47:51,2017-10-25 23:47:51,,Checkin,,,


In [4]:
ci.form.value_counts()

Checkin                    4191
Checkout                   3165
Long Checkin - tba3QU       457
Long Checkout - ljilj8      367
Checkin (quick)             168
Checkin Quick - gu1NmX      164
Checkout Quick - rRzNmu     136
Checkout (quick)            126
                              1
Name: form, dtype: int64

In [5]:
def form_type(v):
    if 'checkin' in v.lower():
        return 'in'
    elif 'checkout' in v.lower():
        return 'out'

    return 'unk'

t = ci.copy()   
t['cico'] = ci.form.apply(form_type)
t.head()
t[t.cico == 'unk']   

t['submit_date'] = pd.to_datetime(t.submit_date)

t.head()


    

,user_id,id,location,feeling,start_date,submit_date,network_id,form,clientid,tablecloth_link,qr_code_link,cico
0,901,,ElCajon,heart eyes,2017-10-25 23:29:23,2017-10-25 23:29:23,,Checkin,,,,in
1,902,,ElCajon,smiling face,2017-10-25 23:30:37,2017-10-25 23:30:37,,Checkin,,,,in
2,81,,ElCajon,heart eyes,2017-10-25 23:32:42,2017-10-25 23:32:42,,Checkin,,,,in
3,1004,,ElCajon,screaming in fear,2017-10-25 23:46:01,2017-10-25 23:46:01,,Checkin,,,,in
4,1009,,ElCajon,smiling face,2017-10-25 23:47:51,2017-10-25 23:47:51,,Checkin,,,,in


In [6]:
def most(x):
    """Select the most common item that isn't empty"""
    try:
        return list(e for e in x.value_counts().index if e and e.strip())[0]
    except AttributeError:
        return list(e for e in x.value_counts().index if e)[0]
    except IndexError:
        return None

t2 = t[(t.user_id.notnull()) & (t.user_id != '')]

g = t2[t2.cico == 'in'].groupby(['user_id',t.submit_date.dt.date])
checkins = g.agg({'submit_date':'min', 'feeling': most, 'location': most})
checkins.columns = ['ci_time', 'ci_feeling', 'location']

g = t2[t2.cico == 'out'].groupby(['user_id',t.submit_date.dt.date])
checkouts = g.agg({'submit_date':'max', 'feeling': most})
checkouts.columns = ['co_time','co_feeling']
checkouts

ci_table = checkins.join(checkouts)
ci_table['session_length'] = ((ci_table.co_time - ci_table.ci_time)/  np.timedelta64(1, 'm')).round(0)
ci_table = ci_table.reset_index().sort_values('ci_time').drop(columns=['submit_date'])
ci_table 

,user_id,ci_time,ci_feeling,location,co_time,co_feeling,session_length
3090,901,2017-10-25 23:29:23,heart eyes,ElCajon,NaT,NaN,NaN
3109,902,2017-10-25 23:30:37,smiling face,ElCajon,NaT,NaN,NaN
2826,81,2017-10-25 23:32:42,heart eyes,ElCajon,NaT,NaN,NaN
9,1004,2017-10-25 23:46:01,screaming in fear,ElCajon,NaT,NaN,NaN
88,1009,2017-10-25 23:47:51,smiling face,ElCajon,NaT,NaN,NaN
51,1008,2017-10-25 23:48:50,smiling face,ElCajon,NaT,NaN,NaN
2783,80,2017-10-26 00:11:11,smiling face,ElCajon,2017-10-26 00:51:02,None,40.0
3116,903,2017-10-26 00:12:56,smiling face,ElCajon,2017-10-26 00:51:59,None,39.0
263,1053,2017-10-27 22:53:59,heart eyes,ImperialBeach,2017-10-27 23:44:22,None,50.0
1107,205,2017-10-27 23:22:05,null,ImperialBeach,2017-10-27 23:47:10,None,25.0


In [8]:
len(ci_table), ci_table.ci_time.min(), ci_table.ci_time.max()

(3343, Timestamp('2017-10-25 23:29:23'), Timestamp('2018-12-16 22:32:03'))

In [9]:
ci = ci.fillna('')
ci

,user_id,id,location,feeling,start_date,submit_date,network_id,form,clientid,tablecloth_link,qr_code_link
0,901,,ElCajon,heart eyes,2017-10-25 23:29:23,2017-10-25 23:29:23,,Checkin,,,
1,902,,ElCajon,smiling face,2017-10-25 23:30:37,2017-10-25 23:30:37,,Checkin,,,
2,81,,ElCajon,heart eyes,2017-10-25 23:32:42,2017-10-25 23:32:42,,Checkin,,,
3,1004,,ElCajon,screaming in fear,2017-10-25 23:46:01,2017-10-25 23:46:01,,Checkin,,,
4,1009,,ElCajon,smiling face,2017-10-25 23:47:51,2017-10-25 23:47:51,,Checkin,,,
5,1008,,ElCajon,smiling face,2017-10-25 23:48:50,2017-10-25 23:48:50,,Checkin,,,
6,80,,ElCajon,smiling face,2017-10-26 00:11:11,2017-10-26 00:11:11,,Checkin,,,
7,903,,ElCajon,smiling face,2017-10-26 00:12:56,2017-10-26 00:12:56,,Checkin,,,
8,81,,ElCajon,,,2017-10-26 00:50:17,,Checkout,,,
9,80,,ElCajon,,,2017-10-26 00:51:02,,Checkout,,,
